In [13]:
import numpy as np
import matplotlib.pyplot as plt
# import google or-tools for solving comvinatorial optimization problem
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model

## Solving Combinatorial Optimization using Google OR-Tools

### Google OR-Tools example: Travelling Salesman Problem

The travelling salesman problem is a popular routing problem that can be solve by combinatorial optimization. Suppose a salesman is trying to go to n number of cities and the distance to travel from each city ot the other is denoted by $d_{ij}$. The goal is to find the minimum distance for the salesman to go to all cities.

**Parameters:**
- number of cities: n 
- $c_{ij}$ : distance from city i to city j

**Decision variables:**
- $x_{ij} = 1$ if the salesman travels from city i to city j

**Dummy variable**
- $u_{i}$ : keeps the sequence of city i visited

**Formulation:**

Objective function:

we minimize the distance that the salesman have to go
$$\text{min} \sum_{i = 1}^n \sum_{j \neq i, j = 1}^n c_{ij}x_{ij}$$

subject to:

Each city needs to be visited: 
$$\sum_{i = 1, i \neq j} x_{ij} = 1 \quad j = 1, \dots, n$$ 
$$\sum_{j = 1, j \neq i} x_{ij} = 1 \quad i = 1, \dots, n$$

The cities need to be visited exactly once so there should not be a loop that visits a subset of cities without visiting all cities exactly once. This means $u_i < u_j$, so $u_j \geq u_i + 1$ if $x_{ij} = 1$, so city i is visited first before city j.

$$u_i - u_j + 1 \leq (n-1)(1-x_{ij}) \quad 2 \leq i \neq j \leq n$$
When $x_{ij} = 0$, the salesman doesn't visit city i to city j, so $u_i - u_j + 1 \leq (n-1)$, which implies that the biggest difference, if city i is visited before city j, is at most n-1. Since we need $u_i < u_j$, then $u_j$ is visited n-1 cities after $u_i$.

Since the starting point is 1, then the order of city i start from 2 to n, the number of cities.
$$2 \leq u_i \leq n$$

(Wikipedia, "Travelling Salesman Problem")
The following data set is taken from GeeksforGeeks ("Traveling Salesman Problem (TSP) Implementation"):

In [53]:
dist_matrix = [[0, 10, 15, 20], 
               [10, 0, 35, 25], 
               [15, 35, 0, 30], 
               [20, 25, 30, 0]]

n = len(dist_matrix)

The first row of dist_matrix contains the distance of city 1 to other cities. For example, the distance from city 1 to city 1 is 0. The distance from city 1 to city 2 is 10, and so on... The second row contains the distance of city 2 to other cities. The third row contains the distance of city 3 to other cities. The fourth row contains the distance of city 4 to other cities. 

The code example for TSP problem using Google OR-Tools is adapted from Google Developers ("Assignment Problem Example")

In [95]:
# Solving MIP problem requires using SCIP
solver = pywraplp.Solver.CreateSolver('SCIP')
x = {}
# all variables are index-0
for i in range(n):
    for j in range(n):
        # x[i, j] can only have 0 and 1 value
        # x[i, j] will be 1 if salesman goes from city i to city j, and 0 otherwise
        x[i, j] = solver.IntVar(0, 1, "")

# define the objective function
objective = []
for i in range(n):
    for j in range(n):
        if j != i:
            objective.append(dist_matrix[i][j]*x[i, j])
solver.Minimize(solver.Sum(objective))

# Each city is visited at least once
for j in range(n):
    solver.Add(solver.Sum(x[i, j] for i in range(n) if i != j) == 1)
for i in range(n):
    solver.Add(solver.Sum(x[i, j] for j in range(n) if j != i) == 1)

# create new decision variables
u = {}
for i in range(1, n):
    u[i] = solver.IntVar(0, 1, "")

# no subloops
# 2 <= i <= n
for i in range(1, n):
     # 2 <= j <= n
     for j in range(1, n):
        if j != i and i != 1 and j != 1:
            solver.Add(u[i] - u[j] + 1 <= (n - 1)*(1 - x[i, j]))
            
status = solver.Solve()
# get the minimum distance that the salesman will travel
print("Minimum distance:", solver.Objective().Value())

Minimum distance: 80.0


In [96]:
route = []
for i in range(n):
    for j in range(n):
        if i != j and x[i, j].solution_value() == 1:
            route.append((i + 1, j + 1))

print(route)

[(1, 3), (2, 4), (3, 1), (4, 2)]


Thus, from the pairings, we can see that the route is city 1-3-4-2-1, which gives the minimum cost 15 + 30 + 25 + 10 = 80. It gives a different route than the website article of traveling salesman problem tsp implementation, but it still gives the same minimum cost.

### Google OR-Tools example: Packing problem 

The knapsack problem is similar to the assignment problem, but we are trying to put as much valuable items as possible to a bag. The problem is not being able to fit everything, so we have to assign which items are more valuable to put into the bag. 

**parameters:**
- number of items: n
- $v_i$ : value of each item i
- $w_i$ : the weight of each item i
- c: maximum capacity of the bag

**decision variables:**
- $x_i = 1$ if we put item i into the bag, and 0 otherwise

**Formulation:**

Objective function:

maximize the total value of the items that goes into the bag
$$ \sum_{i = 1}^n v_i x_i$$
subject to:
$$ \sum_{i = 1}^n w_i x_i \leq c$$
The sum of all items placed into the bag should be less than the maximum capacity of the bag.

(Wikipedia, "Knapsack Problem")

Let there be n = 100 number of items. First, we create the dataset for 100 items. Both values and weights will be randomly generated and the maximum capacity of the bag is 200. 

In [3]:
values = [np.random.randint(1, 100) for i in range(100)]
weights = [np.random.randint(1, 100) for i in range(100)]
c = 200

In [4]:
# create models, variables, and constraints
model = cp_model.CpModel()

# create decision variable
# creates a 0-1 variable 
# x[i] = 1 if item i is in the bag
x = [model.NewBoolVar(f'x-{i}') for i in range(100)]

# define the objective function
# Maximize the total value of the items in the bag
objective = []
for i in range(100):
    objective.append(values[i] *  x[i])
model.Maximize(sum(objective))

# Define the constraitns
# The total weight of items into the bag shouldn't exceed its capacity
total_weights =[]
for i in range(100):
    total_weights.append(weights[i] * x[i])
model.Add(sum(total_weights) <= c)

# solve the model and evaluating solutions
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [5]:
# Print results
items_in_bag = []
# get the items in the bag from the solution of the solver
for i in range(100):
    if solver.Value(x[i]):
        items_in_bag.append(i)
        print('Item :', i, 'Value :', values[i])
# for all items in bag, get the total weight
total_weights = sum(weights[i] for i in items_in_bag)
# for all items in bag, get the total value
total_value = sum(values[i] for i in items_in_bag)
print('Total weight: ', total_weights)
print('Total value: ', total_value)

Item : 1 Value : 27
Item : 2 Value : 57
Item : 12 Value : 25
Item : 31 Value : 24
Item : 34 Value : 60
Item : 47 Value : 78
Item : 48 Value : 48
Item : 49 Value : 88
Item : 56 Value : 77
Item : 64 Value : 85
Item : 69 Value : 73
Item : 70 Value : 90
Item : 76 Value : 67
Item : 98 Value : 82
Total weight:  200
Total value:  881


### References

* “Travelling Salesman Problem.” Wikipedia, Wikimedia Foundation, 3 Mar. 2024, en.wikipedia.org/wiki/Travelling_salesman_problem. (MTZ formulation)
* Singh, Nishant. “Traveling Salesman Problem (TSP) Implementation.” GeeksforGeeks, 11 Nov. 2017, www.geeksforgeeks.org/traveling-salesman-problem-tsp-implementation/
* “Solving an Assignment Problem | OR-Tools.” Google Developers, developers.google.com/optimization/assignment/assignment_example
* “Knapsack Problem.” Wikipedia, 5 Sept. 2020, en.wikipedia.org/wiki/Knapsack_problem